In [1]:
import argparse
import json
import pandas as pd
from vi import Config, Window
from simulation.envs.robot_env import RobotEnvironment
from pathlib import Path
import tempfile
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def run_tree(bt_string: str) -> dict:
    """Run a single behavior tree and return its metrics"""
    # Create a temporary file to store the tree
    with tempfile.NamedTemporaryFile(mode='w', suffix='.xml', delete=False) as tmp:
        tmp.write(bt_string)
        tmp_path = tmp.name

    # Setup and run simulation
    config = Config(radius=25, visualise_chunks=True, window=Window.square(500), 
                   movement_speed=1, duration=1000)
    environment = RobotEnvironment(config=config, bt_path=tmp_path, 
                                 n_agents=10, n_parts=15, task="any", headless=True)
    environment.setup()
    metrics = environment.run()
    # print(metrics)
    
    # Clean up temp file
    Path(tmp_path).unlink()
    
    return metrics

def process_dataset(input_path: str, n_trees: int) -> pd.DataFrame:
    """Process n_trees from the dataset and collect their metrics"""
    # Load dataset
    with open(input_path, 'r') as f:
        dataset = json.load(f)
    

    # Initialize results dataframe and timing tracking
    results = []
    tree_times = []
    start_time = time.time()
    
    # Process trees
    total_trees = min(n_trees, len(dataset))
    for i in range(total_trees):
        tree_start = time.time()
        tree_data = dataset[i]
        
        try:
            # Run the tree and get metrics
            metrics = run_tree(tree_data['tree'])
            
            # Add tree index and metrics to results
            result = {'tree_index': i}
            result.update(metrics)
            results.append(result)
            
            # Track time for this tree
            tree_time = time.time() - tree_start
            tree_times.append(tree_time)
            if len(tree_times) > 25:
                tree_times.pop(0)
            
            # Every 25 trees, print timing estimate
            if (i + 1) % 25 == 0 or i == 0:
                elapsed_time = (time.time() - start_time) / 60
                avg_time_per_tree = sum(tree_times) / len(tree_times)
                trees_remaining = total_trees - (i + 1)
                estimated_time_remaining = (avg_time_per_tree * trees_remaining) / 60
                
                print(f"Trees Processed: {i+1}/{total_trees} - "
                      f"Time Elapsed: {elapsed_time:.1f}m - "
                      f"Remaining Time: {estimated_time_remaining:.1f}m")
            
        except Exception as e:
            print(f"Error processing tree {i}: {str(e)}")
            continue
    
    # Convert results to DataFrame
    return pd.DataFrame(results)

In [4]:
dataset_path = "./data_generation_grammar/datasets/enriched_dataset_b_v2.json"
n_trees = 6000
print(f"Processing {n_trees} trees from {dataset_path}")
# Process the trees and get metrics
results_df = process_dataset(dataset_path, n_trees)

# Print summary statistics
print("\nMetrics Summary:")
print(results_df.describe())

Processing 6000 trees from ./data_generation_grammar/datasets/enriched_dataset_b_v2.json
Trees Processed: 1/2000 - Time Elapsed: 0.0m - Remaining Time: 18.2m
Trees Processed: 25/2000 - Time Elapsed: 0.2m - Remaining Time: 17.9m
Trees Processed: 50/2000 - Time Elapsed: 0.4m - Remaining Time: 14.6m
Trees Processed: 75/2000 - Time Elapsed: 0.6m - Remaining Time: 15.0m
Trees Processed: 100/2000 - Time Elapsed: 0.8m - Remaining Time: 14.5m
Trees Processed: 125/2000 - Time Elapsed: 1.0m - Remaining Time: 14.6m
Trees Processed: 150/2000 - Time Elapsed: 1.2m - Remaining Time: 14.1m
Trees Processed: 175/2000 - Time Elapsed: 1.4m - Remaining Time: 14.1m
Trees Processed: 200/2000 - Time Elapsed: 1.6m - Remaining Time: 13.8m
Trees Processed: 225/2000 - Time Elapsed: 1.8m - Remaining Time: 13.3m
Trees Processed: 250/2000 - Time Elapsed: 1.9m - Remaining Time: 13.5m
Trees Processed: 275/2000 - Time Elapsed: 2.1m - Remaining Time: 13.5m
Trees Processed: 300/2000 - Time Elapsed: 2.3m - Remaining Time:

In [5]:
# Save results to CSV
output_path = "dataset_b_v2_metrics.csv"
results_df.to_csv(output_path, index=False)
print(f"Results saved to {output_path}")

Results saved to dataset_b_v2_metrics.csv


In [6]:
results_df.head()

,tree_index,total_parts_placed,good_parts_correctly_placed,bad_parts_correctly_placed,good_parts_picked_up,bad_parts_picked_up,nest_integrity
0,0,15,15,0,15,5,100
1,1,19,15,0,19,0,100
2,2,0,0,0,6,9,0
3,3,0,0,0,7,10,0
4,4,0,0,0,6,9,0


In [7]:
print(f"Total Trees: {len(results_df)}")

# Get the number of trees that succesfully picked up a good part
n_good_pickups = len(results_df[results_df['good_parts_picked_up'] > 0])
print(f"Trees that picked up good parts: {n_good_pickups}")

# Get the number of trees that succesfully picked up a scrap part
n_bad_pickups = len(results_df[results_df['bad_parts_picked_up'] > 0]) 
print(f"Trees that picked up scrap parts: {n_bad_pickups}")

# Get the number of trees that succesfully placed a good part correctly
n_good_placed = len(results_df[results_df['good_parts_correctly_placed'] > 0])
print(f"Trees that correctly placed good parts: {n_good_placed}")

# Get the number of trees that succesfully placed a scrap part correctly
n_bad_placed = len(results_df[results_df['bad_parts_correctly_placed'] > 0])
print(f"Trees that correctly placed scrap parts: {n_bad_placed}")


Total Trees: 2000
Trees that picked up good parts: 35
Trees that picked up scrap parts: 91
Trees that correctly placed good parts: 4
Trees that correctly placed scrap parts: 1
